Automatic speech recognition (ASR) converts a speech signal to text, mapping a sequence of audio inputs to text outputs. Virtual assistants like Siri and Alexa use ASR models to help users everyday, and there are many other useful user-facing applications like live captioning and note-taking during meetings.

This guide shows how to:
1. Finetune Wav2Vec2 on the MInDS-14 dataset to transcribe audio to text.
2. Use your finetuned model for inference.

# Libraries

In [ ]:
pip install transformers datasets evaluate jiwer

In [ ]:
from datasets import load_dataset, Audio
from transformers import AutoProcessor

# Load Data

In [ ]:
# Load a smaller subset of the MInDS-14 (to experiment on a small dataset first)
minds = load_dataset("PolyAI/minds14", name="en-US", split="train[:100]")

# train-test split 
minds = minds.train_test_split(test_size=0.2)

In [ ]:
# Inspect dataset detail
# NB: focusing on the audio and transcription
# audio: a 1-dimensional array of the speech signal that must be called to load and resample the audio file.
# transcription: the target text.
minds

In [ ]:
# Inspect an example
minds["train"][0]

# Preprocessing

In [ ]:
# Load a Wav2Vec2 processor to process the audio signal
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base")

In [ ]:
# MInDS-14 dataset has a sampling rate of 8000kHz (you can find this information in its dataset card)
# You’ll need to resample the dataset to 16000kHz to use the pretrained Wav2Vec2 model
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))
minds["train"][0]

In [ ]:
# The transcription text contains a mix of upper and lowercase characters
# The Wav2Vec2 tokenizer is only trained on uppercase characters
# ...make sure the text matches the tokenizer’s vocabulary
def uppercase(example):
    return {"transcription": example["transcription"].upper()}

minds = minds.map(uppercase)

In [ ]:
def prepare_dataset(batch):
    # Call the audio column to load and resample the audio file
    audio = batch["audio"]
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["transcription"])
    
    # Extracts the input_values from the audio file and tokenize the transcription column with the processor
    batch["input_length"] = len(batch["input_values"][0])
    return batch

encoded_minds = minds.map(prepare_dataset, remove_columns=minds.column_names["train"], num_proc=4)